History of economic calendar events from https://www.forexfactory.com/calendar

In [6]:
# Import libraries
import pandas as pd
import numpy as np
import regex as re

import requests
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [7]:
# Determine the beginning and end of the date for parsing
date_start = '1/1/2007'
date_end = '3/13/2023'

In [8]:
# Create dataframe for our data
df = pd.DataFrame(columns = ('year', 'day', 'time', 'impact', 'cur', 'text', 'actual', 'forecast', 'previous'))

# Create circle for every day
for i in pd.date_range(start=date_start, end=date_end):
# Create url address and parsing data of current day 
    month = i.month_name().lower()[:3]
    date = month + str(i.day) + '.' + str(i.year)
    url = "https://www.forexfactory.com/calendar?day=" + date
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    html = driver.page_source
    soup = BS(html, "html.parser")
    
#  Correcting the data
    day = soup.find_all('td', class_='calendar__cell calendar__date date')
    time = soup.find_all('td', class_='calendar__cell calendar__time time')
    impact = soup.find_all('div', class_='calendar__impact-icon calendar__impact-icon--screen')
    text = soup.find_all('td', class_='calendar__cell calendar__event event')
    cur = soup.find_all('td', class_='calendar__cell calendar__currency currency')
    actual = soup.find_all('td', class_='calendar__cell calendar__actual actual')
    forecast = soup.find_all('td', class_='calendar__cell calendar__forecast forecast')
    previous = soup.find_all('td', class_='calendar__cell calendar__previous previous')

    day_arr = [i.text.strip()[3:] for i in day]
    time_arr = [i.text.strip() for i in time]
    impact_arr = [re.findall(r'<span class="(\w*)"', str(i))[0] for i in impact]
    text_arr = [i.text.strip() for i in text]
    cur_arr = [i.text.strip() for i in cur]
    actual_arr = [i.text.strip() for i in actual]
    forecast_arr = [i.text.strip() for i in forecast]
    previous_arr = [i.text.strip() for i in previous]

# Create dataframe for current day's data    
    df1 = pd.DataFrame(data=zip(day_arr, time_arr, impact_arr, cur_arr, text_arr, actual_arr, forecast_arr, previous_arr),
                     columns = ('day', 'time', 'impact', 'cur', 'text', 'actual', 'forecast', 'previous'))
    df1 = df1.applymap(lambda x: np.NaN if x.strip() == '' else x)
    
# Add 'year', 'day' and 'time' columns
    df1['year'] = i.year
    df1[['day', 'time']] = df1[['day', 'time']].fillna(method='ffill')
    
# Add current day data to main dataframe
    df = df.append(df1, ignore_index=True)


c:\users\begla\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # Remove the CWD from sys.path while we load stuff.


In [9]:
# Look on our dataframe
df.head()

,year,day,time,impact,cur,text,actual,forecast,previous
0,2022,Jan 16,6:50pm,low,JPY,Core Machinery Orders m/m,3.4%,1.3%,3.8%
1,2022,Jan 16,7:01pm,low,GBP,Rightmove HPI m/m,0.3%,NaN,-0.7%
2,2022,Jan 16,9:00pm,high,CNY,GDP q/y,4.0%,3.3%,4.9%
3,2022,Jan 16,9:00pm,medium,CNY,Retail Sales y/y,1.7%,3.8%,3.9%
4,2022,Jan 16,9:00pm,low,CNY,Fixed Asset Investment ytd/y,4.9%,4.9%,5.2%


In [10]:
# Import data in csv file
df.to_csv("data.csv", index=False)